## Question 1

For the image toys.gif

**A)** Read and display the image.  
**B)** Compute the 2-D FFT of the image and display one full-scaled copy of the magnitude spectrum.  

<img src="toys.gif" width="200" height="200"/>

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image

# Read and display the image
img = Image.open('toys.gif').convert('L')
img_array = np.array(img)
plt.imshow(img_array, cmap='gray')
plt.title('Original Image')
plt.show()

# Compute the 2-D FFT of the image and display the magnitude spectrum
f = np.fft.fft2(img_array)
fshift = np.fft.fftshift(f)
magnitude_spectrum = 20*np.log(np.abs(fshift))
plt.imshow(magnitude_spectrum, cmap='gray')
plt.title('Magnitude Spectrum')
plt.show()

# Apply inverse shift
f_ishift = np.fft.ifftshift(fshift)

# Apply inverse Fourier transform
img_back = np.fft.ifft2(f_ishift)
img_back = np.abs(img_back)

# Display the resulting image
plt.imshow(img_back, cmap='gray')
plt.title('Inverse Fourier Transform result')
plt.show()


## Question 2

For the image pepper.jpg, it is required to study the ringing effect of the Butterworth lowpass filter. Design the following filters and investigate the effect of changing the filter order and cutoff frequency on ringing. You need to display the filtered images and the shape of the filters in the spatial domain. A cross-sectional view of the filters in the spatial domain is required. Make sure to display the figures in a neat and organized way.

| Filter Number | Order | Cutoff Frequency |
|----------|----------|----------|
| 1 | 1 | 50 |
| 2 | 1 | 150 |
| 3 | 5 | 50 |
| 4 | 5 | 150 |

<img src="pepper.jpg" width="200" height="200"/>

In [ ]:
import cv2
from math import sqrt ,exp
def plot_figure(images: list, titles: list, rows: int, columns: int, fig_width=15, fig_height=7):
    fig = plt.figure(figsize=(fig_width, fig_height))
    count = 1
    for image, title in zip(images, titles):
        fig.add_subplot(rows, columns, count)
        count += 1
        plt.imshow(image, 'gray')
        plt.axis('off')
        plt.title(title)
        
def euclidean_dist(point1, point2):
    return sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)

def butterworth_LP(D0: int, image_shape: tuple, order: int):
    base = np.zeros(image_shape[:2])
    rows, cols = image_shape[:2]
    center = (rows/2, cols/2)
    for x in range(cols):
        for y in range(rows):
            base[y, x] = 1 / (1 + (euclidean_dist((y, x), center) / D0) ** (2 * order))
    return base

def display_freq(freq):
    return 20*np.log(np.abs(freq))
      
image = cv2.imread('pepper.jpg', 0)
plot_figure([image], ['pepper.jpg'], 1, 1)

fourier_transform = np.fft.fft2(image)
center_shift = np.fft.fftshift(fourier_transform)

for cutoff_freq , order in zip( [50, 150, 50, 150],[1, 1, 5, 5]):
    BLPF = butterworth_LP(cutoff_freq, image.shape, order)
    filtered = center_shift * BLPF
    ifftshift_filtered = np.fft.ifftshift(filtered)
    final_image = abs(np.fft.ifft2(ifftshift_filtered))
    plot_figure(
        [ display_freq(center_shift), BLPF, display_freq(filtered), final_image],
        [ 'Fourier Transform & Centered Shift', f'BLPF(D0={cutoff_freq}, order={order})',\
         'Filtered Frequence', 'Result'],
        1, 5, 18, 12
    )

## Question 3

For the image prob2_s2012.gif

**A)** Compute and display the magnitude spectrum of the image.     
**B)** Investigate the image in the spatial and frequency domains and determine the type of noise that is corrupting the image.     
**C)** Design the proper filter and use it to restore the original image.       
**D)** Based on the filter in part C, can you display the noise component in the spatial domain? If so, show the image that represents the noise.


<img src="prob2_s2012.gif" width="200" height="200"/>

In [ ]:
import cv2
from math import sqrt ,exp
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image

def gaussian_LP(D0: int, image_shape: tuple):
    base = np.zeros(image_shape[:2])
    rows, cols = image_shape[:2]
    center = (rows/2, cols/2)
    for x in range(cols):
        for y in range(rows):
            base[y,x] = exp((-euclidean_dist((y,x),center)**2)/(2*(D0**2)))
    return base

def ideal_LP(D0: int, image_shape: tuple):
    base = np.zeros(image_shape[:2])
    rows, cols = image_shape[:2]
    center = (rows/2, cols/2)
    for x in range(cols):
        for y in range(rows):
            if D0 >= euclidean_dist((y, x), center):
                base[y, x] = 1
            else:
                base[y, x] = 0
    return base

image_name = 'prob2_s2012.gif'
image = Image.open(image_name)
plot_figure([image], [image_name], 1, 1)
#####
fourier_transform = np.fft.fft2(image)
center_shift = np.fft.fftshift(fourier_transform)
magnitude_spectrum = display_freq(center_shift)
plot_figure([image, magnitude_spectrum], [image_name, f'Magnitude Spectrum'], 1, 2)
#####

cutoff_freq = 75

ilpf = ideal_LP(cutoff_freq, (image.height, image.width))
filtered = center_shift * ilpf
ifftshift_filtered = np.fft.ifftshift(filtered)
final_image = abs(np.fft.ifft2(ifftshift_filtered))

plot_figure([image, final_image],
            [image_name, f'Denoised image with ILPF(D0={cutoff_freq}'], 1, 2)
##
cutoff_freq = 75

glpf = gaussian_LP(cutoff_freq, (image.height, image.width))
filtered = center_shift * glpf
ifftshift_filtered = np.fft.ifftshift(filtered)
final_image = abs(np.fft.ifft2(ifftshift_filtered))

plot_figure([image, final_image],
            [image_name, f'Denoised image with GLPF(D0={cutoff_freq}'], 1, 2)


###
cutoff_freq, order = 75, 2

BLPF = butterworth_LP(cutoff_freq, (final_image.shape[0], final_image.shape[1]), order)
filtered = center_shift * BLPF
ifftshift_filtered = np.fft.ifftshift(filtered)
final_image = abs(np.fft.ifft2(ifftshift_filtered))

plot_figure([image, final_image],
            [image_name, f'Denoised image with GLPF & BLPF(D0={cutoff_freq}, order={order})'], 1, 2)
##

cutoff_freq = 75

glpf = gaussian_LP(cutoff_freq, (image.height, image.width))
filtered = center_shift * (1-glpf)
ifftshift_filtered = np.fft.ifftshift(filtered)
final_image = np.fft.ifft2(ifftshift_filtered)

plot_figure([image, display_freq(final_image)],
            [f'{image_name}', f'extracted Noise with GHPF'], 1, 2)

## Question 4

For the image trucknoise.gif

**A)** Display the magnitude spectrum of the image.     
**B)** Determine the type of noise in the frequency domain.     
**C)** Can you remove the noise? Which type of filter is needed?        
**D)** Display the noise-removed image and compute the PSNR of the image (truck.gif is the original image)


<div style="display: flex;">
    <img src="truck.gif" alt="Image 1" width="200" height="200"/>
    <img src="trucknoise.gif" alt="Image 2" width="200" height="200"/>
</div>


In [ ]:
#B) type of noise is peroiodic with right diagonal noise 
#C) yes ,  with  butterworthLP filter 
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt


def euclidean_dist(point1, point2):
    return sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)


def butterworthLP(D0, imgShape, n):
    base = np.zeros(imgShape[:2])
    rows, cols = imgShape[:2]
    center = (rows / 2, cols / 2)
    for x in range(cols):
        for y in range(rows):
            base[y, x] = 1 / (1 + (euclidean_dist((y, x), center) / D0) ** (2 * n))
    return base


img = cv.imread("trucknoise.png", 0)

fourier_transform = np.fft.fft2(img)
center_shift = np.fft.fftshift(fourier_transform)
magnitude_spectrum = display_freq(center_shift)
plot_figure([image, magnitude_spectrum], [image_name, f'Magnitude Spectrum'], 1, 2)

fourier_transform = np.fft.fft2(img)
center_shift = np.fft.fftshift(fourier_transform)

fourier_noisy = 20 * np.log(np.abs(center_shift))

rows, cols = img.shape
crow, ccol = rows // 2, cols // 2

print("Enter type of noise:- \n1.Vertical Noise\n2.Horizontal Noise\n3.Right Diagonal Noise\n4.Left Diagonal Noise\n")
val = int(input("Enter the Value:- "))

if val == 1:
    # horizontal mask
    center_shift[crow - 4:crow + 4, 0:ccol - 10] = 1
    center_shift[crow - 4:crow + 4, ccol + 10:] = 1
elif val == 2:
    # vertical mask
    center_shift[:crow - 10, ccol - 4:ccol + 4] = 1
    center_shift[crow + 10:, ccol - 4:ccol + 4] = 1
elif val == 3:
    # diagonal-1 mask
    for x in range(0, rows):
        for y in range(0, cols):
            if (x == y):
                for i in range(0, 10):
                    center_shift[x - i, y] = 1
elif val == 4:
    # diagonal-2 mask
    for x in range(0, rows):
        for y in range(0, cols):
            if (x + y == cols):
                for i in range(0, 10):
                    center_shift[x - i, y] = 1

else:
    print("Invalid Input")

filtered = center_shift * butterworthLP(80, img.shape, 10)

f_shift = np.fft.ifftshift(center_shift)
denoised_image = np.fft.ifft2(f_shift)
denoised_image = np.real(denoised_image)

f_ishift_BLPF = np.fft.ifftshift(filtered)
denoised_image_BLPF = np.fft.ifft2(f_ishift_BLPF)
denoised_image_BLPF = np.real(denoised_image_BLPF)

fourier_noisy_noise_removed = 20 * np.log(np.abs(center_shift))

fig = plt.figure(figsize=(8, 8))
ax1 = fig.add_subplot(2, 3, 1)
ax1.title.set_text("Original Image")
ax1.imshow(img, cmap='gray')
ax2 = fig.add_subplot(2, 3, 2)
ax2.imshow(fourier_noisy, cmap='gray')
ax2.title.set_text("Fourier Transform")
ax3 = fig.add_subplot(2, 3, 3)
ax3.imshow(fourier_noisy_noise_removed, cmap='gray')
ax3.title.set_text("Fourier Transform with mask")
ax4 = fig.add_subplot(2, 3, 4)
ax4.imshow(denoised_image, cmap='gray')
ax4.title.set_text("Denoised and unfiltered image")
ax5 = fig.add_subplot(2, 3, 6)
ax5.imshow(denoised_image_BLPF, cmap='gray')
ax5.title.set_text("Denoised and filtered image")

plt.show()




In [5]:
import cv2
import  numpy as np 

# Compute Peak Signal-to-Noise Ratio (PSNR)
original_img = cv2.imread('truck.png', 0)
mse = np.mean((original_img - denoised_image_BLPF) ** 2)
psnr = 20 * np.log10(255 / np.sqrt(mse))
print('Peak Signal-to-Noise Ratio (PSNR):', psnr)

Peak Signal-to-Noise Ratio (PSNR): 24.744088191502847
